In [67]:
import pandas as pd
import numpy as np

In [14]:
# ruta al archivo csv
archivo_temperaturas='temperaturas_globales.csv'
archivo_precipitaciones='precipitaciones_globales.csv'

In [19]:
df_temperaturas = pd.read_csv(archivo_temperaturas, parse_dates=['Fecha'])
df_precipitaciones=pd.read_csv(archivo_precipitaciones,parse_dates=['Fecha'])

In [20]:
df_temperaturas['Fecha'].dtype

dtype('<M8[ns]')

In [ ]:
df_temperaturas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Fecha       24 non-null     datetime64[ns]
 1   Nueva York  24 non-null     int64         
 2   París       24 non-null     int64         
 3   Berlín      24 non-null     int64         
 4   Londres     24 non-null     int64         
 5   Madrid      24 non-null     int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 1.3 KB


In [24]:
df_precipitaciones_ny = pd.read_csv(archivo_precipitaciones, usecols=['Fecha', 'Nueva York'], parse_dates=['Fecha'])

In [26]:
df_precipitaciones_ny.head()

,Fecha,Nueva York
0,2022-01-31,63
1,2022-02-28,78
2,2022-03-31,35
3,2022-04-30,33
4,2022-05-31,80


In [27]:
df_precipitaciones.size

144

In [28]:

size = 1000000

df_chunk = pd.read_csv('precipitaciones_globales.csv', chunksize=size)

In [29]:
for chunk in df_chunk: 

   print(chunk.head()) # Imprime las primeras 5 filas del primer fragmento

   break

        Fecha  Nueva York  París  Berlín  Londres  Madrid
0  2022-01-31          63     78      33       99      82
1  2022-02-28          78     34      76       57      86
2  2022-03-31          35     23      74       52      58
3  2022-04-30          33     75      90       88      65
4  2022-05-31          80     81      85       49      49


## Conectando a base de datos con sqlalquemy

El proceso generalmente implica crear un motor de base de datos y luego usar pd.read_sql() o pd.read_sql_query().

In [30]:
# from sqlalchemy import create_engine
# engine = create_engine('sqlite:///mi_base_de_datos.db')
# pd.read_sql('SELECT * FROM mi_tabla', con=engine)

### Guardar el dataframe en un archivo. Ej en un csv

In [33]:
# Sintaxis básica para guardar en formato CSV:

# df.to_csv('file.csv', index=False)

#Adicionalmente se puede especificat el separador, codificacion (para archivos en otros idiomas) o especificar una ruta para guardarlo 

### Notas adicionales
Para archivos muy grandes, considera leer y procesar los datos en fragmentos o utilizar dtypes más eficientes.

In [35]:
df_temperaturas.shape

(24, 6)

### **Expresiones utiles para rellenar o eliminar filas:**

 Borrar filas con valores nulos:
Es recomendable en un dataset muy grande (millones de registros) donde un 1 % de pérdida no afecte la representatividad.


 reemplazar valores nulos con una medida representativa (media, mediana, moda):
Se recomienda en datasets pequeños o cuando esos registros son críticos 

Valores duplicados:
Si no los quitas, podrías contar la misma entidad dos veces y sesgar tu análisis.

In [ ]:
# .dropna()  # Elimina filas con valores nulos
# .dropna(subset=[…])  # Elimina filas con valores nulos en columnas específicas
# .fillna(value)  # Rellena valores nulos con un valor específico
# .replace(to_replace, value)  # Reemplaza valores específicos
# .fillna(method='ffill')  # Rellena valores nulos con el valor anterior (forward fill)
# .fillna(method='bfill')  # Rellena valores nulos con el valor siguiente (backward fill)
# .duplicated(subset=[...])  # Identifica filas duplicadas basadas en columnas específicas
#.drop_duplicates(subset=[...],keep='first')  # Elimina filas duplicadas, manteniendo la primera ocurrencia

#### Estrategias de imputacion

Cuando un campo está vacío, debemos decidir cómo rellenarlo para no introducir sesgos ni perder información relevante. A continuación, tres métodos muy usados:

1. **Media**: Es conveniente usarla solo si la distribución de tus datos es relativamente simétrica, sin valores extremos que desvíen la media.
    - Cuando no usar media para imputar valores:

        Ahora, si estás trabajando con ingresos mensuales de personas en una encuesta social y la mayoría gana entre $500 000 y $1 000 000, pero hay algunas personas con ingresos extremadamente altos, de más de $10 000 000; si faltan 15 valores y los completas con la media (~$1 500 000), estarías:

        Introduciendo un valor que no representa al grupo típico.

        Aumentando artificialmente la dispersión.

        Posiblemente alterando resultados de modelos predictivos o promedios de referencia.

        En este caso, la media está sesgada por los valores extremos y sería más apropiado usar la mediana, ya que representa mejor el centro de una distribución asimétrica.

2. **Mediana**: util cuando los datos tienen una distribución sesgada o con valores atípicos, ya que la mediana no se ve afectada por extremos como sí ocurre con la media.

3. **Moda**: Util cuando se trabaja con variables categoricas o variables numericas con pocos valores distintos (ej. numero de hijos o nivel educativo codificado)




|Estrategia| Mejor Para| Cuando evitarla|
|---|---|---|
|Media|Variables numéricas simétricas y sin outliers |Si hay valores extremos|
|Moda|Variables numéricas sesgadas o con outliers|Si necesitas mantener la media original|
|Moda|Variables categóricas o discretas|Si hay múltiples modas o muchas categorías|

#### Normalizacion de datos

Normalizar es ajustar la escala de las variables para que estén en rangos comparables o tengan distribuciones estándar, evitando que las magnitudes de unos atributos “dominen” sobre otros en el modelado.
- Ejemplo: Escalado Min–Max

Este método se encarga de reescalar los valores al rango [0, 1] según:

X'=(X-Xmin)/Xmax-Xmin

X es el valor original.

Xmin y Xmax son el valor mínimo y máximo de la variable.

X' es el valor normalizado (reescala X al rango [0, 1]).

¿Cuándo usarlo?

Al preparar gráficos o dashboards donde dos variables con unidades distintas (por ejemplo, temperatura en °C y precipitación en mm) deben mostrarse juntas en el mismo eje.

En análisis de correlación visual, para evitar que una variable con rango amplio “aplane” la escala de la otra.

#### **Exactitud**
Mide qué tan cercanos están los valores a la realidad o a la fuente original

Detection:
- Chequeo puntual: toma una muestra aleatoria de filas y compara los valores con la base de datos transaccional, un PDF de facturas o un sistema de origen.
- Validaciones automatizadas: por ejemplo, si tu columna monto_venta supera cierto umbral histórico, marca esos registros para revisión manual.

#### **Integridad**
Mide el porcentaje de datos presentes frente a los que se esperaría tener.

Umbral util:
- 10-30% datos faltantes: Hay que justificar la estrategia (borrar vs. imputar) y documentarla.
- mas del 30 % datos faltantes: Reevaluar si la columna es fiable o si conviene buscar otra fuente.

#### **Consistencia**
Mide la ausencia de contradicciones internas o violaciones a reglas de negocio. Por ejemplo:
- Rango de fechas inválido: fin < inicio.
- Valores fuera de dominio: puntajes > 100 % o negativos.
- Violación de unicidad: IDs duplicados donde deben ser únicos.

In [57]:
meses_calurosos_madrid = df_temperaturas[(df_temperaturas['Madrid'] > 25)]

### Mascaras en Numpy

Las máscaras en Numpy no solo se limitan a filtrar datos mediante condiciones; también se pueden emplear para aplicaciones más específicas, incluyendo manipulaciones estructurales de matrices y la creación de arrays especializados. 

Funciones utiles:
- np.triu: devuelve la parte triangular superior de un array. Util para trabajar con matrices en operaciones algebraicas, especialmente en la manipulación de matrices simétricas o en la solución de sistemas de ecuaciones lineales.
- np.ones_like: crea un nuevo array con la misma forma y tipo que un array dado, llenándolo todo con unos. útil para crear máscaras iniciales o arrays de pesos cuando trabajamos con operaciones que involucran matrices o arrays de dimensiones específicas.

#### np.where en pandas

np.where(condición, valor_si_True, valor_si_False) devuelve un array (o Serie) donde, para cada elemento, evalúa la condición y elige uno de los dos valores especificados. Cuando lo integramos en Pandas, aprovechamos la velocidad de NumPy para aplicar reglas en toda una columna al mismo tiempo.

Es muy util, ya que incluye la eficiencia de numpy en pandas. Se puede aplicar el metodo np.where... a un dataframe ya que pandas esta construido en numpy y hereda muchos de sus atributos.
Es especialmente util para especificar exactamente el resultado que queremos si una condicion se cumple o no se cumple, por lo que permite personalizacion

### Tecnicas Avanzadas

.apply(func) aplica una función a cada elemento de una Serie, lo que permite una gran flexibilidad en la manipulación de datos. La funcion puede ser una funcion lambda, lo que permite una gran facilidad en la syntaxis

In [63]:
df_temperaturas['Clasificación_Madrid'] = df_temperaturas['Madrid'].apply(lambda x: 'Caluroso' if x > 25 else 'Fresco')



Otras funciones avanzadas incluyen agregación, fusión y transformaciones de datos. permiten realizar análisis más sofisticados y obtener insights más profundos


1. Agrupacion: .groupby() es fundamental para realizar cálculos estadísticos sobre subconjuntos de datos, permitiendo comparar grupos o realizar operaciones específicas por categoría. Metodos luego de agruparlos incluyen: *.sum(), .mean(), .median(), .count(), .max(), .min(), .transform(), .filter()* 


In [94]:
# Categorizando las temperaturas
df_temperaturas['Rango_Temperatura'] = pd.cut(df_temperaturas['Madrid'], bins=[-10, 10, 20, 30, 40], labels=["Baja", "Moderada", "Alta", "Muy Alta"])
# Contando días por rango de temperatura
dias_por_rango_temperatura = df_temperaturas.groupby('Rango_Temperatura')
dias_por_rango_temperatura[['Madrid','Nueva York']].max()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_15432\2791340733.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dias_por_rango_temperatura = df_temperaturas.groupby('Rango_Temperatura')


,Madrid,Nueva York
Rango_Temperatura,,
Baja,6,28
Moderada,20,28
Alta,30,33
Muy Alta,33,32


2. funcion pd.melt() se utiliza para "despivotar" o transformar un DataFrame de un formato ancho a un formato largo, haciendo que cada variable se extienda a través de las columnas.

In [97]:
# pd.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)

df_temperaturas_nuevoformato = df_temperaturas.melt(id_vars=['Fecha','Clasificación_Madrid','Rango_Temperatura'], var_name='Ciudad', value_name='Temperatura')



In [103]:
# Transformaciones con .groupby()

# Calculamos la temperatura media por ciudad

media_por_ciudad = df_temperaturas_nuevoformato.groupby('Ciudad')['Temperatura'].transform('mean')


# Restamos esta media de cada registro de temperatura para centrar los datos

df_temperaturas_nuevoformato['Temperatura_Centrada'] = df_temperaturas_nuevoformato['Temperatura'] - media_por_ciudad


# Mostramos las primeras filas para verificar los resultados

df_temperaturas_nuevoformato.head()


,Fecha,Clasificación_Madrid,Rango_Temperatura,Ciudad,Temperatura,Temperatura_Centrada
0,2022-01-31,Fresco,Alta,Nueva York,2,-14.583333
1,2022-02-28,Fresco,Alta,Nueva York,26,9.416667
2,2022-03-31,Fresco,Baja,Nueva York,20,3.416667
3,2022-04-30,Caluroso,Muy Alta,Nueva York,32,15.416667
4,2022-05-31,Caluroso,Alta,Nueva York,18,1.416667


In [104]:
# USO DE FILTER: Definimos un umbral para la desviación estándar de la temperatura

umbral_variabilidad = 5


# Filtramos ciudades con una variabilidad de temperatura superior al umbral

ciudades_con_alta_variabilidad = df_temperaturas_nuevoformato.groupby('Ciudad').filter(lambda x: x['Temperatura'].std() > umbral_variabilidad)


print("Ciudades con alta variabilidad de temperatura:")

print(ciudades_con_alta_variabilidad['Ciudad'].unique())

Ciudades con alta variabilidad de temperatura:
['Nueva York' 'París' 'Berlín' 'Londres' 'Madrid']


3. Pivot tables

Son una herramienta excelente para resumir, analizar, explorar y presentar tus datos. Pandas facilita la creación de tablas pivote a partir de DataFrames, permitiéndote ver la relación entre dos dimensiones de datos de manera compacta. Son especialmente útiles para análisis de datos exploratorios y reportes.

In [105]:
#  Crear una tabla pivote
# pivot = df.pivot_table(values='valor', index='fila', columns='columna', aggfunc='mean')

In [106]:
# Primero, asegurémonos de que el DataFrame tiene una columna 'Año' extraída de 'Fecha'

df_temperaturas_nuevoformato['Año'] = df_temperaturas_nuevoformato['Fecha'].dt.year



# Crear la tabla pivote

tabla_pivote_maximas = df_temperaturas_nuevoformato.pivot_table(values='Temperatura', index='Año', columns='Ciudad', aggfunc='max')


print("Tabla Pivote de Temperaturas Máximas Anuales por Ciudad:")

print(tabla_pivote_maximas)

Tabla Pivote de Temperaturas Máximas Anuales por Ciudad:
Ciudad  Berlín  Londres  Madrid  Nueva York  París
Año                                               
2022        33       31      33          32     31
2023        34       34      30          33     33


In [107]:
# Otro ejemplo: Si estamos interesados en comparar no solo las temperaturas máximas sino también las medias en un formato mensual para una ciudad específica, como Nueva York,

# Primero, asegurémonos de que el DataFrame tiene una columna 'Mes' extraída de 'Fecha'

df_temperaturas_nuevoformato['Mes'] = df_temperaturas_nuevoformato['Fecha'].dt.month

# Crear la tabla pivote para Nueva York con temperaturas medias y máximas

tabla_pivote_nuevayork = df_temperaturas_nuevoformato[df_temperaturas_nuevoformato['Ciudad'] == 'Nueva York'].pivot_table(values='Temperatura',index='Mes', aggfunc={'Temperatura': ['mean', 'max']})


print("Tabla Pivote de Temperaturas Medias y Máximas Mensuales en Nueva York:")

print(tabla_pivote_nuevayork)

Tabla Pivote de Temperaturas Medias y Máximas Mensuales en Nueva York:
     max  mean
Mes           
1     25  13.5
2     26  10.5
3     20  15.0
4     32  25.5
5     19  18.5
6     11   9.5
7      8   7.0
8     24  13.5
9     28  15.5
10    28  24.5
11    33  26.5
12    32  19.5


#### Fusion y concatenacion de datos

Permite analizar fuentes que necesitan ser unidos para un análisis más completo. pd.merge() y pd.concat() son las mas usadas

1. pd.merge()

In [108]:
## Fusionar DataFrames en una clave común
# df_fusionado = pd.merge(df1, df2, on='clave_comun')

In [110]:
#Ejemplo de fusion de datos:

# Asegurando que los datos estén en el formato correcto

df_temperaturas['Fecha'] = pd.to_datetime(df_temperaturas['Fecha'])

df_precipitaciones['Fecha'] = pd.to_datetime(df_precipitaciones['Fecha'])
# Una vez que ambos están en formato datetime, procedemos con el merge
# Combinar df_temperaturas y df_precipitaciones

df_combinado = pd.merge(df_temperaturas, df_precipitaciones, on='Fecha', suffixes=('_temp', '_prec'))

2. pd.concat() concatena DataFrames o Series a lo largo de un eje particular. Es útil para combinar datos que tienen el mismo esquema. Purpose: Stack DataFrames vertically (rows) or horizontally (columns). Works more like “gluing” DataFrames together, not joining by key.

#### Filtrado con isin()

permite seleccionar filas de un DataFrame o Series donde un valor o conjunto de valores específicos, aparecen en una columna (o índice en el caso de Series). Es particularmente útil cuando quieres filtrar tu DataFrame para incluir sólo ciertos registros.

In [111]:
#Ejemplo:
# A partir de nuestro df, creamos un nuevo df que contenga los registros de Londres y Nueva York.

temperaturas_isin = df_temperaturas_nuevoformato[df_temperaturas_nuevoformato['Ciudad'].isin(['Londres', 'Nueva York'])]

### Funciones Lambda

Son funciones anónimas definidas en una sola línea. En Pandas, las funciones lambda son útiles para aplicar operaciones rápidas o personalizadas a los datos, especialmente dentro de métodos como .apply(), .transform(), y otros que aceptan funciones como argumentos.

**Transformaciones vs. agregaciones**

Mientras que las operaciones de agregación con .groupby() reducen la dimensión de los datos al producir un valor escalar por grupo, las transformaciones mantienen la misma longitud que el DataFrame o Serie original, permitiéndote realizar operaciones complejas a nivel de grupo sin perder la estructura de tus datos

#### Funciones universales de numpy

Las ufuncs, son funciones de Numpy que operan en arrays de Numpy elemento por elemento, facilitando la realización de operaciones matemáticas y lógicas de manera eficiente y con sintaxis simplificado. Son "universales" en el sentido de que pueden manejar argumentos de arrays de diferentes tamaños y dimensiones (broadcasting) con una única llamada a la función, mejorando significativamente la legibilidad y la eficiencia del código.

**Operaciones de ejemplo**:
- Operaciones aritméticas básicas (suma, resta, multiplicación y similares)
- Funciones matemáticas (seno, coseno, exponencial, logaritmo, etcétera)
- Operaciones de comparación y lógicas
- Funciones estadísticas (media, mediana, varianza, entre otras)

**Importancia de las ufuncs**

Las ufuncs son fundamentales en el ecosistema de Numpy debido a su eficiencia y flexibilidad. Permiten realizar operaciones complejas sobre arrays de manera intuitiva y con menos código que si se utilizaran bucles for, mejorando tanto la legibilidad como el rendimiento del código. 